In [ ]:
!pip install -U sentence-transformers

In [83]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import datasets
from sentence_transformers import SentenceTransformer

# Load Data

In [3]:
# data = datasets.load_dataset('json',data_files=r'dataset/train_data.json')
# test = datasets.load_dataset('json',data_files=r'dataset/test_data.json')
train = pd.read_json(r'dataset/train_data.json')
test = pd.read_json(r'dataset/test_data.json')

# Encode Sentence & label

## Encode Input

In [4]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-mpnet-base-v2')

In [5]:
embeddings = encoder.encode(train['caption_string'])

## Encode Output
one hot encoding

In [6]:
train['tag_string'][0]

'1girl, areola_slip, areolae, arm_under_breasts, bangs, black_bodysuit, black_gloves, blush, bodysuit, breasts, closed_mouth, covered_navel, earrings, fingerless_gloves, gem, gloves, jewelry, large_breasts, looking_at_viewer, pyra_(xenoblade), red_eyes, red_hair, short_hair, short_sleeves, sideboob, sidelocks, solo, steam, sweat, swept_bangs, tiara, xenoblade_chronicles_(series), xenoblade_chronicles_2'

In [7]:
tag_list = open(r'dictionaries/tag_dict.txt').read().splitlines()

In [8]:
train['tags'] = train['tag_string'].str.split(', ')

In [9]:
from sklearn import preprocessing
mlb = preprocessing.MultiLabelBinarizer(classes=tag_list)
mlb.fit([list(tag_list)])

MultiLabelBinarizer(classes=['1girl', 'solo', 'long_hair', 'breasts', 'blush',
                             'looking_at_viewer', 'smile', 'short_hair',
                             'open_mouth', 'bangs', 'blue_eyes',
                             'multiple_girls', 'blonde_hair', 'skirt',
                             'brown_hair', 'large_breasts', 'simple_background',
                             'black_hair', 'eyebrows_visible_through_hair',
                             'thighhighs', 'hair_ornament', 'hat', 'red_eyes',
                             'gloves', 'shirt', 'touhou', '1boy', 'dress',
                             'white_background', 'original', ...])

In [10]:
labels = mlb.transform(train.tags)
labels

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

# Model

## Just A Fucking GradientBoost

In [ ]:
!pip install xgboost

In [11]:
from sklearn.datasets import make_multilabel_classification
import xgboost as xgb
import numpy as np

X, y = make_multilabel_classification(
    n_samples=32, n_classes=len(tag_list), n_labels=2, random_state=0
) # dummy data shape
X.shape, y.shape

((32, 20), (32, 11389))

In [12]:
embeddings.shape, labels.shape

((19950, 768), (19950, 11389))

In [13]:
clf = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity=2)
clf.fit(embeddings[:5], labels[:5])

[17:28:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "verbose" } are not used.



KeyboardInterrupt: 

In [11]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
forest = RandomForestClassifier(random_state=69420, verbose=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)

In [13]:
multi_target_forest.fit(embeddings, labels)

In [ ]:
out = multi_target_forest.predict(sentence_embeddings[0].reshape(1, -1))

In [ ]:
prediction = clf.predict(sentence_embeddings)

In [ ]:
multi_target_forest.fit(sentence_embeddings[:100], labels[:100])

In [ ]:
out = multi_target_forest.predict(sentence_embeddings[0].reshape(1, -1))

In [164]:
embeddings = []
for sentence in tqdm(test["caption_string"]):
    embeddings.append(encoder.encode(sentence))

test["embeddings"] = embeddings
test

100%|██████████| 150/150 [00:02<00:00, 55.14it/s]


,caption_string,tag_string,embeddings
0,"With her mesmerizing gaze and ethereal wings, ...","1girl, breasts, closed_mouth, gloves, horns, l...","[0.062214088, 0.0003957237, -0.025598288, -0.0..."
1,Kisaragi Ai's art features Garma Zabi from Gun...,"1boy, adjusting_hair, bow, bowtie, brown_eyes,...","[0.039543997, -0.018483877, 0.0033783775, -0.0..."
2,Rojer18's art depicts Oozora Hiro from Danball...,"1boy, ahoge, belt, blue_hair, closed_mouth, cr...","[0.011929578, 0.0033304796, -0.023017503, 0.02..."
3,"Amidst the chaos of Danganronpa, Criis-chan ca...","2girls, :d, ahoge, arm_up, bangs, black_shirt,...","[0.02401699, 0.006202389, -0.026183875, 0.0457..."
4,The artwork is a solo depiction of the charact...,"1boy, ^_^, ahoge, blush, closed_eyes, fate/gra...","[0.04863312, -0.0014078199, -0.014670438, 0.05..."
...,...,...,...
145,The artwork features the character Bokuto Kout...,"1boy, artist_name, bangs, black_shirt, collarb...","[0.050111346, -0.06240129, -0.0120949885, 0.03..."
146,"""Too_mizuguchi's artwork portrays Oswald Chest...","1boy, bags_under_eyes, batman_(series), black_...","[0.007011963, -0.006353568, 0.0055657285, 0.02..."
147,Denjinq's artistic prowess comes to life with ...,"1boy, 5girls, aqua_hair, ass, bangs, black_hai...","[0.04270737, -0.013710473, -0.025662424, 0.027..."
148,The artwork features a single female character...,"1girl, :o, bangs, black_shirt, boots, bronya_z...","[0.07322672, 0.006603488, -0.0020600723, 0.053..."


In [165]:
X_test = np.vstack(test['embeddings'].to_numpy())

In [167]:
preds = clf.predict(X_test)

In [168]:
pred_tags = decode(preds)

In [169]:
temp = [x for x in pred_tags]
test['pred_tags'] = temp
test['pred_tags']

0                                     (1boy, male_focus)
1      (1girl, solo, long_hair, breasts, blue_eyes, c...
2      (1girl, solo, looking_at_viewer, smile, open_m...
3        (1girl, solo, looking_at_viewer, weapon, sword)
4             (1girl, solo, looking_at_viewer, outdoors)
                             ...                        
145                             (solo, 1boy, male_focus)
146    (solo, gloves, 1boy, male_focus, fate_(series)...
147                                         (solo, 1boy)
148                                         (undertale,)
149                                      (weapon, mecha)
Name: pred_tags, Length: 150, dtype: object

In [172]:
def calc_accuracy(df):
    df['tags'] = df['tag_string'].split(', ')
    correct_count = len(set(df['tags']).intersection(df['pred_tags']))
    # incorrect_count = len(data_point['tags']) - correct_count
    accuracy = correct_count / len(df['tags']) * 100
    return accuracy

In [174]:
test

,caption_string,tag_string,embeddings,pred_tags
0,"With her mesmerizing gaze and ethereal wings, ...","1girl, breasts, closed_mouth, gloves, horns, l...","[0.062214088, 0.0003957237, -0.025598288, -0.0...","(1boy, male_focus)"
1,Kisaragi Ai's art features Garma Zabi from Gun...,"1boy, adjusting_hair, bow, bowtie, brown_eyes,...","[0.039543997, -0.018483877, 0.0033783775, -0.0...","(1girl, solo, long_hair, breasts, blue_eyes, c..."
2,Rojer18's art depicts Oozora Hiro from Danball...,"1boy, ahoge, belt, blue_hair, closed_mouth, cr...","[0.011929578, 0.0033304796, -0.023017503, 0.02...","(1girl, solo, looking_at_viewer, smile, open_m..."
3,"Amidst the chaos of Danganronpa, Criis-chan ca...","2girls, :d, ahoge, arm_up, bangs, black_shirt,...","[0.02401699, 0.006202389, -0.026183875, 0.0457...","(1girl, solo, looking_at_viewer, weapon, sword)"
4,The artwork is a solo depiction of the charact...,"1boy, ^_^, ahoge, blush, closed_eyes, fate/gra...","[0.04863312, -0.0014078199, -0.014670438, 0.05...","(1girl, solo, looking_at_viewer, outdoors)"
...,...,...,...,...
145,The artwork features the character Bokuto Kout...,"1boy, artist_name, bangs, black_shirt, collarb...","[0.050111346, -0.06240129, -0.0120949885, 0.03...","(solo, 1boy, male_focus)"
146,"""Too_mizuguchi's artwork portrays Oswald Chest...","1boy, bags_under_eyes, batman_(series), black_...","[0.007011963, -0.006353568, 0.0055657285, 0.02...","(solo, gloves, 1boy, male_focus, fate_(series)..."
147,Denjinq's artistic prowess comes to life with ...,"1boy, 5girls, aqua_hair, ass, bangs, black_hai...","[0.04270737, -0.013710473, -0.025662424, 0.027...","(solo, 1boy)"
148,The artwork features a single female character...,"1girl, :o, bangs, black_shirt, boots, bronya_z...","[0.07322672, 0.006603488, -0.0020600723, 0.053...","(undertale,)"


In [ ]:
test['accuracy'] = test.apply(calc_accuracy)
test

In [55]:
test_tags_encode = [le.transform(tags.split(", ")) for tags in tqdm(test["tag_string"])]
test_tags_encoded = torch.stack([one_hot_encode(x, len(all_tags)) for x in test_tags_encode])

100%|██████████| 150/150 [00:01<00:00, 117.51it/s]


In [56]:
def token_accuracy(predict, label):
  correct = 0
  index = np.where(label == 1)[0]
  for idx in index:
    if predict[idx] == label[idx]:
      correct += 1

  len_ = len(index)
  if len_ == 0:
    raise ValueError("Label Can't be all  0")
  return correct / len_

In [57]:
def overall_accuracy(predict, label):
  return (np.array(predict) == np.array(label)).sum()/len(label)

In [58]:
accs = []
for idx, predict in enumerate(prediction):
  accs.append(token_accuracy(predict, test_tags_encoded.numpy()[idx]))

In [59]:
sum(accs)/len(accs)

0.14337228824248877

In [60]:
decode_sequence(prediction[1]), decode_sequence(test_tags_encoded.numpy()[1]), test['caption_string'][1]

(array(['1boy', 'male_focus', 'solo'], dtype='<U59'),
 array(['1boy', 'adjusting_hair', 'bow', 'bowtie', 'brown_eyes', 'child',
        'flower', 'formal', 'gundam', 'male_focus', 'mobile_suit_gundam',
        'purple_eyes', 'short_hair', 'smile', 'solo', 'suit', 'vest',
        'younger'], dtype='<U59'),
 "Kisaragi Ai's art features Garma Zabi from Gundam, a young boy with short hair and brown eyes adjusting his hair and bowtie while sporting a flower and a smile in a purple-eyed, formal solo pose with a suit, vest, and bow.")

## It's A typical Neural Network

In [1]:
from fastai.text.all import *
import torch.nn as nn

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [11]:
train_data = torch.tensor(embeddings).float()
train_labels = torch.tensor(labels).float()
train_data.shape, train_labels.shape

(torch.Size([19950, 768]), torch.Size([19950, 11389]))

In [13]:
dataset = torch.utils.data.TensorDataset(train_data, train_labels)

In [14]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [15]:
batch_size = 64
train_dataloader = DataLoaders.from_dsets(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoaders.from_dsets(val_dataset, batch_size=batch_size, shuffle=False)

In [20]:
from fastai.learner import Learner

In [52]:
class MultiLabelClassification(torch.nn.Module):
    def __init__(self, n_inputs, n_classes, optim, loss):
        super().__init__()
        self.optim = 
        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(in_features=n_inputs, out_features=n_inputs),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=n_inputs, out_features=n_inputs),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=n_inputs, out_features=n_inputs),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(in_features=n_inputs, out_features=n_classes),
            torch.nn.Sigmoid(),
        )

    def fit(self, data_loader, epochs, validation_data=None):

        for epoch in range(epochs):
            running_loss = self._train_iteration(data_loader)
            val_loss = None
            if validation_data is not None:
                y_hat = self(validation_data['X'])
                val_loss = self.lossFct(input=y_hat, target=validation_data['y']).detach().numpy()
                print('[%d] loss: %.3f | validation loss: %.3f' %
                  (epoch + 1, running_loss, val_loss))
            else:
                print('[%d] loss: %.3f' %
                  (epoch + 1, running_loss)
                  
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [63]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [118]:
target = torch.ones([10, 64], dtype=torch.float32)
output = torch.full([10, 64], 1.5)
print(output.shape)

criterion = torch.nn.BCEWithLogitsLoss()  # w/o weight
print(criterion(output, target))

weight = torch.rand(10, 1)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([5]))  # w/ weight
print(criterion(output, target))

torch.Size([10, 64])
tensor(0.2014)
tensor(1.0071)


In [122]:
loss_func = nn.BCEWithLogitsLoss()

In [123]:
loss_func(
    torch.Tensor([[0, 1, 0], [0, 1, 0]]), torch.Tensor([[1, 1, 0], [0, 0, 1]])
)

tensor(0.7332)

In [141]:
loss_func_weighted(
    torch.Tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    torch.Tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
)

tensor(24.7846)

In [142]:
loss_func_weighted(
    torch.Tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    torch.Tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
)

tensor(53.9589)

In [134]:
model = MultiLabelClassification(embeddings.shape[1],labels.shape[1]).to(device)
weights = FloatTensor([1000])
loss_func_weighted = nn.BCEWithLogitsLoss(pos_weight=weights)
model

MultiLabelClassification(
  (layer1): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=768, out_features=768, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=768, out_features=768, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=768, out_features=11389, bias=True)
    (1): Sigmoid()
  )
)

In [143]:
for epoch in tqdm(range(3)):
    for batch in tqdm(train_dataset):
        model.train()
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        output = model(x).to(device)
        optimizer.zero_grad()
        loss = loss_func(output, y).to(device)
        loss.backward()
        optimizer.step()
    if epoch % 2 == 0:
        model.eval()
        loss_list = []
        for batch in val_dataset:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            output = model(x).to(device)
            loss = loss_func(output, y).to(device)
            loss_list.append(loss.cpu().detach().numpy())
        print(np.array(loss_list).mean())

 33%|███▎      | 1/3 [00:35<01:11, 35.90s/it]

0.9727103


100%|██████████| 3/3 [01:42<00:00, 34.10s/it]

0.9727103


In [145]:
def token_accuracy(predict, label):
    correct = 0
    index = np.where(label == 1)[0]
    for idx in index:
        if predict[idx] == label[idx]:
            correct += 1

    len_ = len(index)
    if len_ == 0:
        raise ValueError("Label Can't be all  0")
    return correct / len_

In [ ]:
def overall_accuracy(predict, label):
  return (np.array(predict) == np.array(label)).sum()/len(label)

In [167]:
out = model(torch.Tensor(embeddings[:2]).to(device))
out = torch.as_tensor((out - 0.5) > 0, dtype=torch.float)
out

tensor([[1., 1., 0.,  ..., 1., 0., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.]], device='cuda:0')

In [168]:
token_accuracy(out[0], labels[0])

0.48484848484848486

In [179]:
from sklearn import metrics
metrics.f1_score(labels[0],out[0].cpu(),average='macro')

0.335081161501017

In [172]:
len(list(np.where(out[0].cpu() == 1)[0]))

5712

In [169]:
mlb.inverse_transform(out.cpu())

[('1girl',
  'solo',
  'breasts',
  'looking_at_viewer',
  'open_mouth',
  'bangs',
  'skirt',
  'brown_hair',
  'simple_background',
  'eyebrows_visible_through_hair',
  'thighhighs',
  'hair_ornament',
  'shirt',
  '1boy',
  'dress',
  'white_background',
  'long_sleeves',
  'bow',
  'navel',
  '2girls',
  'brown_eyes',
  'cleavage',
  'twintails',
  'medium_breasts',
  'bare_shoulders',
  'school_uniform',
  'very_long_hair',
  'jewelry',
  'green_eyes',
  'blue_hair',
  'panties',
  'closed_mouth',
  'monochrome',
  'kantai_collection',
  'swimsuit',
  'comic',
  'weapon',
  'yellow_eyes',
  'ponytail',
  'full_body',
  'purple_hair',
  'pink_hair',
  'braid',
  'flower',
  'silver_hair',
  'ahoge',
  'hair_bow',
  'upper_body',
  ':d',
  'short_sleeves',
  'white_shirt',
  'pantyhose',
  'heart',
  'bikini',
  'male_focus',
  'pleated_skirt',
  'red_hair',
  'small_breasts',
  'lying',
  'detached_sleeves',
  'food',
  'one_eye_closed',
  'frills',
  'white_legwear',
  'multicolor

In [162]:
def decode_sequence(prediction):
  prediction = list(np.where(prediction == 1)[0])
  return mlb.inverse_transform([prediction])

In [147]:
token_accuracy(np.array([0,1,0,0]),np.array([0,1,1,0]))

0.5

In [71]:
from fastai.optimizer import OptimWrapper

from torch import optim
from functools import partial

optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [72]:
loss_func = nn.BCEWithLogitsLoss()

In [58]:
learner = Learner(
    dls=train_dataloader,
    model=model,
    loss_func=loss_func,
    opt_func=optimizer,
    metrics=partial(accuracy_multi, thresh=0.5),
).to("cuda")

In [ ]:
to

In [59]:
learner.fit(3, lr_max=1e-3)

AttributeError: 'MultiLabelClassification' object has no attribute 'fit'

In [ ]:
epochs = 1000
batch_size = 1000
devices = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = MultiLabelClassification(sentence_embeddings.shape[1], tags_encoded.numpy().shape[1]).to(devices)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
import math
batch_count = math.ceil(sentence_embeddings.shape[0] / batch_size)

In [ ]:
for epoch in tqdm(range(epochs)):
  for batch_idx in range(batch_count):
    model.train()
    model = model.to(devices)
    end = 10 * (batch_idx + 1)
    start = end - 10
    try: # in case it exceed index range
      sliced = sentence_embeddings[start:end] 
      y = tags_encoded[start:end]
    except: # do this instead
      sliced = sentence_embeddings[start:] 
      y = tags_encoded[start:]

    x = torch.from_numpy(sliced).to(devices)
    y = y.to(devices)
    optimizer.zero_grad()
    output = model(x).to(devices)

    loss = criterion(output, y).to(devices)
    loss.backward()
    optimizer.step()
  
  if epoch % 2 == 0:
    with torch.no_grad():
      model.eval()
      model = model.to('cpu')
      y = tags_encoded[:1000]
      output = model(torch.from_numpy(sentence_embeddings[:1000]))
      output = torch.as_tensor((output - 0.5) > 0, dtype=torch.float) # process output to class
      token_accs = []
      for idx, predict in enumerate(output):
        token_accs.append(token_accuracy(predict, y.numpy()[idx]))
      print(f'Train Accuracy: {sum(token_accs)/len(token_accs)} Batchs Loss {loss}')